In [1]:
import yaml
from pathlib import Path

In [2]:
links_path = Path("links_transmision.yml")
locations_path = Path("locations.yaml")

In [3]:
with locations_path.open("r") as f:
    locations_data = yaml.safe_load(f)
valid_locations = set(locations_data.keys())

In [4]:
with links_path.open("r") as f:
    links_data = yaml.safe_load(f)

In [5]:
links_dict = links_data.get("links", {})


In [8]:
new_links_dict = {}
for key, value in links_dict.items():
    try:
        loc1, loc2 = key.split(",")
    except ValueError:
        print(f"Clave inválida: {key}")
        continue
    if not loc1.startswith("TAP") and loc1 in valid_locations:
        loc1 = f"S_E_{loc1}"
    if not loc2.startswith("TAP") and loc2 in valid_locations:
        loc2 = f"S_E_{loc2}"
    new_key = f"{loc1},{loc2}"
    new_links_dict[new_key] = value

new_links_data = {"links": new_links_dict}


In [9]:
output_path = Path("links_transmision_modificado.yml")
with output_path.open("w") as f:
    yaml.dump(new_links_data, f, sort_keys=False)
print(f"Archivo modificado guardado como: {output_path}")


Archivo modificado guardado como: links_transmision_modificado.yml


In [8]:
import yaml
import csv


with open("locations.yaml", "r") as f:
    data = yaml.safe_load(f)
    
locations = data.get("locations", {})

with open("locations.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "longitude", "latitude"])
    for name, attrs in locations.items():
        coords = attrs.get("coordinates")
        if coords and len(coords) == 2:
            writer.writerow([name, coords[0], coords[1]])


In [9]:
import folium
import pandas as pd

df = pd.read_csv("locations.csv")
m = folium.Map(location=[-33.5, -70.5], zoom_start=6)

for _, row in df.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["name"]
    ).add_to(m)

m.save("mapa.html")


In [11]:
import folium
import pandas as pd
import geopandas as gpd

# === MAPA BASE CON NODOS ===
df = pd.read_csv("locations.csv")
m = folium.Map(location=[-33.5, -70.5], zoom_start=6)

for _, row in df.iterrows():
    if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=row["name"],
            icon=folium.Icon(color="blue", icon="bolt", prefix='fa')
        ).add_to(m)

# === LÍNEAS DE TRANSMISIÓN ===
shapefile = "/home/aninotna/magister/tesis/justh2_pipeline/data/generadoras/Línea_de_Transmisión.shp"
gdf = gpd.read_file(shapefile)

for _, row in gdf.iterrows():
    geometry = row.geometry
    if geometry and geometry.geom_type == "LineString":
        coords = [(lat, lon) for lon, lat in geometry.coords]  # invertir orden para folium

        # Extraer nombre del tramo
        tramo = str(row.get("TRAMO", ""))
        partes = [p.strip().replace(" ", "_").replace("-", "_") for p in tramo.split("-")]
        if len(partes) < 2:
            continue
        from_node, to_node = partes[0], partes[1]
        nombre_tramo = f"{from_node} → {to_node}"

        # Extraer tensión si está disponible
        tension = row.get('TENSION_KV', 'N/A')

        # Agregar línea al mapa con nombre y tensión
        folium.PolyLine(
            coords,
            color="red",
            weight=2,
            popup=f"<b>{nombre_tramo}</b><br>Tensión: {tension} kV"
        ).add_to(m)

# === GUARDAR MAPA FINAL ===
m.save("mapa_completo.html")
print("✅ Mapa generado con nodos y líneas: mapa_completo.html")


✅ Mapa generado con nodos y líneas: mapa_completo.html
